# Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
import warnings

# Игнорировать все предупреждения
warnings.filterwarnings('ignore')

In [3]:
df_class = pd.read_csv('classification.csv')
df_reg = pd.read_csv('regression.csv')

Датасет для классификации (Air Quality & Health Impact Analysis):
* **RecordID:** Уникальный идентификатор, присваиваемый каждой записи
* **AQI:** Индекс качества воздуха, показывающий, насколько загрязнен воздух в настоящее время или насколько загрязненным он, по прогнозам, станет в будущем
* **PM10**:  Концентрация твердых частиц диаметром менее 10 микрометров (μg/m³)
* **PM2_5**: Концентрация твердых частиц диаметром менее 2,5 микрометров (μg/m³)
* **NO2**: Концентрация диоксида азота (ppb)
* **SO2**: Концентрация диоксида серы (ppb)
* **O3**: Концентрация озона (ppb)
* **Temperature**: Температура в градусах Цельсия (°C)
* **Humidity**: Процент влажности (%)
* **WindSpeed**: Скорость ветра (m/s)
* **RespiratoryCases**: Количество зарегистрированных респираторных случаев.
* **CardiovascularCases**: Количество зарегистрированных сердечно-сосудистых случаев
* **HospitalAdmissions**: Количество зарегистрированных случаев госпитализации
* **Target Variable: HealthImpactClass**

Датасет для регрессии (Electrity Prices):
* **DateTime**: дата и время
* **Holiday**: название праздника, если день нерабочий день
* **HolidayFlag**: целое число, 1, если день нерабочий день, ноль в противном случае
* **DayOfWeek**: целое число (0-6), 0 понедельник, день недели
* **WeekOfYear**: текущая неделя в течение года, начинающегося с этой даты
* **Day integer**: день
* **Month integer**: месяц
* **Year integer**: год
* **PeriodOfDay**: период суток
* **ForecastWindProduction**: прогнозируемая мощность ветра на этот период
* **SystemLoadEA**: национальный прогноз нагрузки на этот период
* **SMPEA**: прогноз цен на данный период
* **ORKTemperature**: фактическая температура
* **ORKWindspeed**: фактическая скорость ветра
* **CO2Intensity**: фактическая интенсивность выбросов CO2 в произведенной электроэнергии (г/кВт*ч)
* **ActualWindProduction**: фактическая нагрузка на национальную систему за этот период
* **SystemLoadEP2**: фактическая цена за данный период времени, прогнозируемое значение.
* **Target Variable: SystemLoadEP2**

## Создание бейзлайна

### Классификация

In [5]:
def simple_classification(df):
  X_class = df.drop(['HealthImpactClass','HealthImpactScore','RecordID'], axis=1)
  y_class = df['HealthImpactClass']

  X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

  return X_train_class, X_test_class, y_train_class, y_test_class

In [6]:
X_train, X_test, y_train, y_test = simple_classification(df_class)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy_class = accuracy_score(y_test, predictions)
print(f"Точность логистической регрессии: {accuracy_class:.4f}")

Точность логистической регрессии: 0.8572


### Регрессия

In [8]:
def simple_regression(df):
  X_reg = df.drop(['SMPEP2', 'DateTime','Holiday'], axis=1, errors='ignore')
  y_reg = df['SMPEP2']

  for col in X_reg.columns:
      X_reg[col] = pd.to_numeric(X_reg[col], errors='coerce')

  y_reg = pd.to_numeric(y_reg, errors='coerce')

  X_reg = X_reg.fillna(0)
  y_reg = y_reg.fillna(0)

  X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

  return X_train_reg, X_test_reg, y_train_reg, y_test_reg

In [9]:
X_train, X_test, y_train, y_test = simple_regression(df_reg)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка линейной регресии: {mse_reg:.4f}")

Среднеквадратичная ошибка линейной регресии: 687.1066


## Улучшение бейзлайна

### Классификация

In [10]:
def upgraded_classification(df):
  df = df.drop(columns=["RecordID"], errors="ignore")

  low_PM2_5 = df["PM2_5"].quantile(0.25)
  strong_PM2_5 = df["PM2_5"].quantile(0.75)
  df["LowPM2_5"] = (df["PM2_5"] <= low_PM2_5).astype(int)
  df["StrongPM2_5"] = (df["PM2_5"] >= strong_PM2_5).astype(int)

  low_PM10 = df["PM10"].quantile(0.25)
  strong_PM10 = df["PM10"].quantile(0.75)
  df["LowPM10"] = (df["PM10"] <= low_PM10).astype(int)
  df["StrongPM10"] = (df["PM10"] >= strong_PM10).astype(int)


  low_AQI = df["AQI"].quantile(0.25)
  strong_AQI = df["AQI"].quantile(0.75)
  df["LowAQI"] = (df["AQI"] <= low_AQI).astype(int)
  df["StrongAQI"] = (df["AQI"] >= strong_AQI).astype(int)

  df = df.drop(columns=["RespiratoryCases", "CardiovascularCases","WindSpeed","Temperature","Humidity",'PM2_5'], axis=1)

  df = df[[
      "HealthImpactClass",
      "LowAQI",
      "LowPM2_5",
      "LowPM10",
      "StrongPM2_5",
      "PM10",
      "O3",
      "StrongAQI",
      "AQI",
      "HealthImpactScore"
  ]]

  X_class = df.drop(['HealthImpactClass','HealthImpactScore'], axis=1)
  y_class = df['HealthImpactClass']

  X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

  scaler = StandardScaler()

  X_train_class_scaled = scaler.fit_transform(X_train_class)

  X_test_class_scaled = scaler.transform(X_test_class)

  return X_train_class_scaled, X_test_class_scaled, y_train_class, y_test_class

In [11]:
X_train, X_test, y_train, y_test = upgraded_classification(df_class)

In [13]:
# Поиск оптимальных гиперпараметров
model = LogisticRegression(random_state=42)

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['liblinear', 'saga'],
    'l1_ratio': [0, 0.5, 1]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучший результат:", grid_search.best_score_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Лучшие параметры: {'C': 10, 'l1_ratio': 0, 'penalty': 'l1', 'solver': 'saga'}
Лучший результат: 0.8760776207270673


In [14]:
model = LogisticRegression(C= 10, l1_ratio= 0, penalty= 'l1', solver='saga',random_state = 42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy_class = accuracy_score(y_test, predictions)
print(f"Точность логистической регрессии: {accuracy_class:.4f}")

Точность логистической регрессии: 0.8813


### Регрессия

In [15]:
def upgraded_regression(df):
  df['ForecastWindProduction'] = pd.to_numeric(df['ForecastWindProduction'], errors='coerce')
  df['SystemLoadEA'] = pd.to_numeric(df['SystemLoadEA'], errors='coerce')
  df['SMPEA'] = pd.to_numeric(df['SMPEA'], errors='coerce')
  df['ORKTemperature'] = pd.to_numeric(df['ORKTemperature'], errors='coerce')
  df['ORKWindspeed'] = pd.to_numeric(df['ORKWindspeed'], errors='coerce')
  df['CO2Intensity'] = pd.to_numeric(df['CO2Intensity'], errors='coerce')
  df['ActualWindProduction'] = pd.to_numeric(df['ActualWindProduction'], errors='coerce')
  df['SystemLoadEP2'] = pd.to_numeric(df['SystemLoadEP2'], errors='coerce')
  df['SMPEP2'] = pd.to_numeric(df['SMPEP2'], errors='coerce')

  df = df.drop(["DateTime","Holiday"],axis = 1)

  df = df.dropna()

  df = df[df['SMPEP2'] > 0]
  df = df[df['SMPEP2'] != 1000]

  X_reg = df.drop(['SMPEP2'], axis=1, errors='ignore')
  y_reg = df['SMPEP2']

  X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

  scaler = StandardScaler()
  X_train_reg_scaled = scaler.fit_transform(X_train_reg)
  X_test_reg_scaled = scaler.transform(X_test_reg)

  return X_train_reg_scaled, X_test_reg_scaled, y_train_reg, y_test_reg

In [16]:
X_train, X_test, y_train, y_test = upgraded_regression(df_reg)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка линейной регресии: {mse_reg:.4f}")

Среднеквадратичная ошибка линейной регресии: 674.0785


Стандартный бейзлайн:
* Точность логистической регрессии: 0.8572
* Среднеквадратичная ошибка линейной регресии: 687.1066

Улучшенный байзлайн:
* Точность логистической регрессии: 0.8813
* Среднеквадратичная ошибка  линейной регресии: 674.0785

***Итог:*** Улучшение предварительной обработки данных и настроек моделей привело к положительному эффекту:

Логистическая регрессия: точность выросла с 0.8572 до 0.8813, что свидетельствует о более качественной классификации.

Линейная регрессия: значение MSE снизилось с 687.1066 до 674.0785, что указывает на улучшение точности предсказаний.

Таким образом, улучшенный бейзлайн демонстрирует заметное повышение качества как классификационной, так и регрессионной моделей.

## Имплементация алгоритма

#### Реализация логистической регрессии

In [17]:
class My_Logistic_Regression:

    def __init__(self, learning_rate=0.0001, n_iterations=10000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None


    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))


    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db


    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return np.where(y_predicted >= 0.5, 1, 0)

#### Реализация линейной регрессии

In [18]:
class My_Linear_Regression:

    def __init__(self, learning_rate=0.0001, n_iterations=10000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None


    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_pred = np.dot(X, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db


    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [19]:
X_train, X_test, y_train, y_test = simple_classification(df_class)

model = My_Logistic_Regression()
model.fit(X_train, y_train)
y_pred_class = model.predict(X_test)

accuracy_class = accuracy_score(y_test, y_pred_class)
print(f"Точность логистической регрессии: {accuracy_class:.4f}")

Точность логистической регрессии: 0.8274


In [20]:
X_train, X_test, y_train, y_test = simple_regression(df_reg)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = My_Linear_Regression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка (MSE) линейной регресии: {mse_reg:.4f}")

Среднеквадратичная ошибка (MSE) линейной регресии: 1251.2675


In [21]:
X_train, X_test, y_train, y_test = upgraded_classification(df_class)

model = My_Logistic_Regression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy_class = accuracy_score(y_test, predictions)
print(f"Точность логистической регрессии: {accuracy_class:.4f}")

Точность логистической регрессии: 0.7701


In [22]:
X_train, X_test, y_train, y_test = upgraded_regression(df_reg)

model = My_Linear_Regression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка (MSE) линейной регресии: {mse_reg:.4f}")

Среднеквадратичная ошибка (MSE) линейной регресии: 1269.9549


**Стандартный бейзлайн:**

*Библиотечная реализация:*
* Точность логистической регрессии: 0.8572
* Среднеквадратичная ошибка линейной регресии: 687.1066

*Имплементация алгоритма:*
* Точность логистической регрессии: 0.8274
* Среднеквадратичная ошибка линейной регресии: 1251.2675


**Улучшенный байзлайн:**

*Библиотечная реализация:*
* Точность логистической регрессии: 0.8813
* Среднеквадратичная ошибка линейной регресии: 674.0785

Имплементация алгоритма:
* Точность логистической регрессии: 0.7701
* Среднеквадратичная ошибка линейной регресии: 1269.9549


Итог: В ходе выполнения лабораторной работы были построены и проанализированы как стандартные, так и улучшенные бейзлайны для задач классификации и регрессии. Рассматривались два подхода: использование библиотечных реализаций и самостоятельная имплементация алгоритмов.

Библиотечные реализации оказались более устойчивыми и эффективными: улучшение данных или гиперпараметров действительно привело к повышению качества.

Собственная имплементация показала худшие результаты по сравнению с библиотечной и оказалась чувствительной к улучшению бейзлайна - качество даже снизилось.
